In [109]:
import numpy
import pandas as pd
import numpy as np
import tensorflow
import keras
from matplotlib import pyplot
import seaborn
import itertools
import datetime 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer



%matplotlib inline

In [110]:
#LOADING DATA INTO A DATAFRAME
occurrences = pd.read_excel("../../../dataset/Ocorrências-1.xlsx")
occurrences

/home/higoreller/Development/pos_machine_learning/venv/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell F221384 is marked as a date but the serial value -736954.9663425926 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


,rai,data,naturezas,bairro_cidade,recurso,tr,obm_escala,obm_afeto,qualificacao,sexo
0,5957792,2018-04-01 00:08:00,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->PROBLEM...,VILA ITATIAIA - GOIÂNIA,(null),(null),(null),(null),(null),(null)
1,5957802,2018-04-01 00:07:00,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->CRISE H...,VILA OLIVEIRA - APARECIDA DE GOIÂNIA,UR-226,00:12:08.560000,7º BBM - APARECIDA DE GOIÂNIA,7º BBM - APARECIDA DE GOIÂNIA,(null),(null)
2,5957806,2018-04-01 00:05:00,BUSCA E SALVAMENTO (4)->PESSOAS (403)->RETIRAD...,LUCILENE - SANTA HELENA DE GOIÁS,UR-171,00:11:21.955000,3ª CIBM - SANTA HELENA,3ª CIBM - SANTA HELENA,(null),(null)
3,5957839,2018-04-01 00:23:00,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,SÍTIOS SANTA LUZIA - APARECIDA DE GOIÂNIA,UR-140,00:25:05.944000,7º BBM - APARECIDA DE GOIÂNIA,7º BBM - APARECIDA DE GOIÂNIA,(null),(null)
4,5957849,2018-04-01 00:12:00,INCÊNDIO URBANO (6)->VEÍCULOS (610)->VEÍCULO D...,ZONA RURAL - NIQUELÂNDIA,ABTS-07,00:03:08.962000,6ª CIBM - NIQUELÂNDIA,6ª CIBM - NIQUELÂNDIA,(null),(null)
...,...,...,...,...,...,...,...,...,...,...
822072,25960823,2022-08-09 00:23:00,RESGATE (3)->INTOXICAÇÃO EXÓGENA (306)->MEDICA...,AEROVIÁRIO - GOIÂNIA,UR-276,00:08:46.442000,2º BBM - SETOR AEROVIÁRIO,2º BBM - SETOR AEROVIÁRIO,(null),(null)
822073,25960831,2022-08-09 00:25:00,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,ZONA RURAL - INHUMAS,ASA-168,(null),7ª CIBM - INHUMAS,7ª CIBM - INHUMAS,(null),(null)
822074,25960831,2022-08-09 00:25:00,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,ZONA RURAL - INHUMAS,UR-262,(null),7ª CIBM - INHUMAS,7ª CIBM - INHUMAS,(null),(null)
822075,25960846,2022-08-08 09:10:00,AÇÕES PREVENTIVAS (2)->DIVERSAS (201)->POSICIO...,VILA PARAISO I - SANTO ANTÔNIO DO DESCOBERTO,UR-235,00:05:00,3º PBM STO. A. DO DESCOBERTO,3º PBM STO. A. DO DESCOBERTO,(null),(null)


In [111]:
#DATA PREPARATION

#A coluna referente ao "sexo" tem influência na previsão, entretanto após a remoção dos valores
#"(null)" e "NÃO-INFORMADO" de sua coluna o número de linhas cai de 822077 para 424345, perdendo
#com isso um volume muito grande de dados. Após a avaliação da rede sem a coluna será feita uma
#análise com a coluna referente ao sexo para ver o impacto sobre a figura de mérito escolhida.

occurrences = occurrences.drop(["rai", "obm_afeto", "qualificacao", "sexo"], axis=1) 
occurrences

,data,naturezas,bairro_cidade,recurso,tr,obm_escala
0,2018-04-01 00:08:00,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->PROBLEM...,VILA ITATIAIA - GOIÂNIA,(null),(null),(null)
1,2018-04-01 00:07:00,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->CRISE H...,VILA OLIVEIRA - APARECIDA DE GOIÂNIA,UR-226,00:12:08.560000,7º BBM - APARECIDA DE GOIÂNIA
2,2018-04-01 00:05:00,BUSCA E SALVAMENTO (4)->PESSOAS (403)->RETIRAD...,LUCILENE - SANTA HELENA DE GOIÁS,UR-171,00:11:21.955000,3ª CIBM - SANTA HELENA
3,2018-04-01 00:23:00,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,SÍTIOS SANTA LUZIA - APARECIDA DE GOIÂNIA,UR-140,00:25:05.944000,7º BBM - APARECIDA DE GOIÂNIA
4,2018-04-01 00:12:00,INCÊNDIO URBANO (6)->VEÍCULOS (610)->VEÍCULO D...,ZONA RURAL - NIQUELÂNDIA,ABTS-07,00:03:08.962000,6ª CIBM - NIQUELÂNDIA
...,...,...,...,...,...,...
822072,2022-08-09 00:23:00,RESGATE (3)->INTOXICAÇÃO EXÓGENA (306)->MEDICA...,AEROVIÁRIO - GOIÂNIA,UR-276,00:08:46.442000,2º BBM - SETOR AEROVIÁRIO
822073,2022-08-09 00:25:00,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,ZONA RURAL - INHUMAS,ASA-168,(null),7ª CIBM - INHUMAS
822074,2022-08-09 00:25:00,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,ZONA RURAL - INHUMAS,UR-262,(null),7ª CIBM - INHUMAS
822075,2022-08-08 09:10:00,AÇÕES PREVENTIVAS (2)->DIVERSAS (201)->POSICIO...,VILA PARAISO I - SANTO ANTÔNIO DO DESCOBERTO,UR-235,00:05:00,3º PBM STO. A. DO DESCOBERTO


In [113]:
#REMOVING NULL VALUES
occurrences.loc[pd.isnull(occurrences["data"])]
occurrences.loc[pd.isnull(occurrences["naturezas"])]
occurrences.loc[pd.isnull(occurrences["bairro_cidade"])]
occurrences.loc[pd.isnull(occurrences["tr"])]
occurrences.loc[pd.isnull(occurrences["obm_escala"])]

occurrences = occurrences.loc[occurrences["bairro_cidade"] != "(null)"]
occurrences = occurrences.loc[occurrences["recurso"] != "(null)"]
occurrences = occurrences.loc[occurrences["tr"] != "(null)"]
occurrences = occurrences.loc[occurrences["obm_escala"] != "(null)"]

# occurrences = occurrences.loc[occurrences["sexo"] != "(null)"]
# occurrences = occurrences.loc[occurrences["sexo"] != "NÃO-INFORMADO"]

occurrences

,data,naturezas,bairro_cidade,recurso,tr,obm_escala
1,2018-04-01 00:07:00.000,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->CRISE H...,VILA OLIVEIRA - APARECIDA DE GOIÂNIA,UR-226,00:12:08.560000,7º BBM - APARECIDA DE GOIÂNIA
2,2018-04-01 00:05:00.000,BUSCA E SALVAMENTO (4)->PESSOAS (403)->RETIRAD...,LUCILENE - SANTA HELENA DE GOIÁS,UR-171,00:11:21.955000,3ª CIBM - SANTA HELENA
3,2018-04-01 00:23:00.000,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,SÍTIOS SANTA LUZIA - APARECIDA DE GOIÂNIA,UR-140,00:25:05.944000,7º BBM - APARECIDA DE GOIÂNIA
4,2018-04-01 00:12:00.000,INCÊNDIO URBANO (6)->VEÍCULOS (610)->VEÍCULO D...,ZONA RURAL - NIQUELÂNDIA,ABTS-07,00:03:08.962000,6ª CIBM - NIQUELÂNDIA
5,2018-04-01 00:16:00.000,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->ACIDENT...,ZONA RURAL - CATALÃO,UR-181,00:04:16.701000,10º BBM - CATALÃO
...,...,...,...,...,...,...
822067,2022-08-08 09:10:00.000,AÇÕES PREVENTIVAS (2)->DIVERSAS (201)->POSICIO...,SETOR LESTE VILA NOVA - GOIÂNIA,ASA-159,00:00:00,1º BBM - SETOR CENTRAL
822068,2022-08-09 00:05:00.000,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->OUTRO /...,CIDADE VERA CRUZ - APARECIDA DE GOIÂNIA,UR-231,00:15:14.007000,7º BBM - APARECIDA DE GOIÂNIA
822069,2022-08-07 21:09:29.730,AÇÕES PREVENTIVAS (2)->DIVERSAS (201)->POSICIO...,SETOR LESTE VILA NOVA - GOIÂNIA,ASA-159,1900-01-01 00:00:30.270000,1º BBM - SETOR CENTRAL
822072,2022-08-09 00:23:00.000,RESGATE (3)->INTOXICAÇÃO EXÓGENA (306)->MEDICA...,AEROVIÁRIO - GOIÂNIA,UR-276,00:08:46.442000,2º BBM - SETOR AEROVIÁRIO


In [114]:
#TRANSFORMING "data" COLUMN INTO NEW COLUMNS "dia" e "periodo". ALSO TRANSFORMING "tr" COLUMN INTO "tempo_resposta" COLUMN

def day_name(timestamp):
    weekdays = ('Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo')
    return weekdays[timestamp.weekday()]

def period_of_day(timestamp):
    period = ("Madrugada", "Matutino", "Vespertino", "Noturno")
    # Madrugada 00:00 às 05:59
    # Matutino 06:00 às 11:59
    # Vespertino 12:00 às 17:59
    # Noturno 18:00 às 23:59
    if timestamp.hour >= 0 and timestamp.hour < 6:
        return period[0]
    elif timestamp.hour >= 6 and timestamp.hour < 12:
        return period[1]
    elif timestamp.hour >= 12 and timestamp.hour < 18:
        return period[2]
    elif timestamp.hour >= 18 and timestamp.hour < 24:
        return period[3]



def response_time(response_time):
    # Muito rápido 0 a 10 minutos
    # Rápido 10 a 15 minutos
    # Médio 15 a 20 minutos
    # Longo 20 a 30 minutos
    # Muito longo 30 a 45 minutos
    # Extremamente longo > 45 minutos

    response_time_metric = ("Muito rápido", "Rápido", "Médio", "Longo", "Muito longo", "Extremamente longo")

    if type(response_time) is datetime.time:

        total_time_in_minutes = response_time.hour*60 + response_time.minute + response_time.second/60

        if total_time_in_minutes >= 0 and total_time_in_minutes <= 10:
            return response_time_metric[0]
        elif total_time_in_minutes > 10 and total_time_in_minutes <= 15:
            return response_time_metric[1]
        elif total_time_in_minutes > 15 and total_time_in_minutes <= 20:
            return response_time_metric[2]
        elif total_time_in_minutes > 20 and total_time_in_minutes <= 30:
            return response_time_metric[3]
        elif total_time_in_minutes > 30 and total_time_in_minutes <= 45:
            return response_time_metric[4]
        elif total_time_in_minutes > 45:
            return response_time_metric[5]
        
#Lembrar de remover os valores que não são datetime.time do occurrences["tr"]
occurrences.loc[:, "dia"] = occurrences["data"].apply(day_name)
occurrences.loc[:, "periodo"] = occurrences["data"].apply(period_of_day)
occurrences.loc[:, "tempo_resposta"] = occurrences["tr"].apply(response_time)

In [120]:
#REMOVING "DATA" AND "TR" COLUMNS

occurrences = occurrences.drop(["data", "tr"], axis=1)

#Removing None values
occurrences = occurrences.dropna()
occurrences = occurrences.mask(occurrences.eq('None')).dropna()
occurrences = occurrences.astype(str)

occurrences

,naturezas,bairro_cidade,recurso,obm_escala,dia,periodo,tempo_resposta
1,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->CRISE H...,VILA OLIVEIRA - APARECIDA DE GOIÂNIA,UR-226,7º BBM - APARECIDA DE GOIÂNIA,Domingo,Madrugada,Rápido
2,BUSCA E SALVAMENTO (4)->PESSOAS (403)->RETIRAD...,LUCILENE - SANTA HELENA DE GOIÁS,UR-171,3ª CIBM - SANTA HELENA,Domingo,Madrugada,Rápido
3,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,SÍTIOS SANTA LUZIA - APARECIDA DE GOIÂNIA,UR-140,7º BBM - APARECIDA DE GOIÂNIA,Domingo,Madrugada,Longo
4,INCÊNDIO URBANO (6)->VEÍCULOS (610)->VEÍCULO D...,ZONA RURAL - NIQUELÂNDIA,ABTS-07,6ª CIBM - NIQUELÂNDIA,Domingo,Madrugada,Muito rápido
5,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->ACIDENT...,ZONA RURAL - CATALÃO,UR-181,10º BBM - CATALÃO,Domingo,Madrugada,Muito rápido
...,...,...,...,...,...,...,...
822064,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...,LOTEAMENTO LEÃO - CATALÃO,UR-225,10º BBM - CATALÃO,Segunda-feira,Noturno,Muito longo
822067,AÇÕES PREVENTIVAS (2)->DIVERSAS (201)->POSICIO...,SETOR LESTE VILA NOVA - GOIÂNIA,ASA-159,1º BBM - SETOR CENTRAL,Segunda-feira,Matutino,Muito rápido
822068,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->OUTRO /...,CIDADE VERA CRUZ - APARECIDA DE GOIÂNIA,UR-231,7º BBM - APARECIDA DE GOIÂNIA,Terça-feira,Madrugada,Médio
822072,RESGATE (3)->INTOXICAÇÃO EXÓGENA (306)->MEDICA...,AEROVIÁRIO - GOIÂNIA,UR-276,2º BBM - SETOR AEROVIÁRIO,Terça-feira,Madrugada,Muito rápido


In [121]:
forecasters = occurrences.iloc[:, [1, 2, 3, 4, 5, 6]].values
nature = occurrences.iloc[:, [0]]
nature

,naturezas
1,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->CRISE H...
2,BUSCA E SALVAMENTO (4)->PESSOAS (403)->RETIRAD...
3,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...
4,INCÊNDIO URBANO (6)->VEÍCULOS (610)->VEÍCULO D...
5,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->ACIDENT...
...,...
822064,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CARRO...
822067,AÇÕES PREVENTIVAS (2)->DIVERSAS (201)->POSICIO...
822068,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->OUTRO /...
822072,RESGATE (3)->INTOXICAÇÃO EXÓGENA (306)->MEDICA...


In [122]:
forecasters

array([['VILA OLIVEIRA - APARECIDA DE GOIÂNIA', 'UR-226',
        '7º BBM - APARECIDA DE GOIÂNIA', 'Domingo', 'Madrugada',
        'Rápido'],
       ['LUCILENE - SANTA HELENA DE GOIÁS', 'UR-171',
        '3ª CIBM - SANTA HELENA', 'Domingo', 'Madrugada', 'Rápido'],
       ['SÍTIOS SANTA LUZIA - APARECIDA DE GOIÂNIA', 'UR-140',
        '7º BBM - APARECIDA DE GOIÂNIA', 'Domingo', 'Madrugada', 'Longo'],
       ...,
       ['CIDADE VERA CRUZ - APARECIDA DE GOIÂNIA', 'UR-231',
        '7º BBM - APARECIDA DE GOIÂNIA', 'Terça-feira', 'Madrugada',
        'Médio'],
       ['AEROVIÁRIO - GOIÂNIA', 'UR-276', '2º BBM - SETOR AEROVIÁRIO',
        'Terça-feira', 'Madrugada', 'Muito rápido'],
       ['VILA PARAISO I - SANTO ANTÔNIO DO DESCOBERTO', 'UR-235',
        '3º PBM STO. A. DO DESCOBERTO', 'Segunda-feira', 'Matutino',
        'Muito rápido']], dtype=object)

In [131]:
#DATA ENCODING
label_encoder_forecasters = LabelEncoder()

#I Had memory problems so I've tried 100.000 rows first.
one_hot_encoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 2, 3, 4, 5])], remainder='passthrough')
forecasters = one_hot_encoder.fit_transform(forecasters[:100000]).toarray()
forecasters


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])